# Load libraries and data

In [1]:
import os
import sys
import hydra
from hydra.core.hydra_config import HydraConfig
from omegaconf import DictConfig, OmegaConf
from tqdm import tqdm
import pandas as pd
import numpy as np
import random
import submitit
import time
from tqdm import tqdm

# use datatime for dates outside of the pd.datetime bounds
from datetime import datetime



In [2]:
ids = [
    6337610,
    6502171,
    6335046,
    6338200,
    6338160,
    6123710,
    6503851,
    6229100,
    6335351,
    6123350,
    6139682,
    6503280,
    6503500,
    6731600,
    6338120,
    6934571,
    6338161,
    6503855,
    6335160,
    6139790,
    6502151,
    6233350,
    6233100,
    6321100,
    6342521,
    6604220,
    6243400,
    6338150,
    6503281,
    6337504,
    6335045,
    6503201,
    6335081,
    6119200,
    6854590,
    6337050,
    6503180,
    6123160,
    6503351,
    6335360,
    6503301,
    6855409,
    6136200,
    6503300,
    6233520,
  ]
len(ids)

45

In [3]:
# get the basin information as provided by the grdc
grdc_info = pd.read_excel('/data/compoundx/GRDC_runoff/GRDC_Stations.xlsx')
grdc_info = grdc_info.rename(columns={"grdc_no": "id"})
grdc_info = grdc_info.set_index("id")

In [4]:
grdc_info

,wmo_reg,sub_reg,river,station,country,lat,long,area,altitude,d_start,...,m_start,m_end,m_yrs,m_miss,t_start,t_end,t_yrs,lta_discharge,r_volume_yr,r_height_yr
id,,,,,,,,,,,,,,,,,,,,,
1104150,1,1041,"CHELIF, OUED",SIDI BELATAR,DZ,36.020000,0.270000,43750.0,2.0,1978.0,...,1976.0,1977.0,2.0,0.0,1976,2001,26,15.027,0.473891,10.831805
1104200,1,1041,"MINA, OUED",OUED EL-ABTAL,DZ,35.500000,0.680000,6635.0,205.0,NaN,...,1976.0,1978.0,3.0,0.0,1976,1978,3,2.578,0.0813,12.253174
1104300,1,1041,"GHIOU, OUED",AMMI MOUSSA,DZ,35.870000,1.120000,2398.0,140.0,NaN,...,1976.0,1979.0,4.0,0.0,1976,1979,4,3.195,0.100758,42.017314
1104450,1,1040,"MAZAFRAN, OUED",FER A CHEVAL,DZ,36.670000,2.820000,1912.0,10.0,1979.0,...,1976.0,1978.0,3.0,0.0,1976,1995,20,3.379,0.10656,55.732293
1104480,1,1040,"BOU DOUAOU, OUED",KEDDARA,DZ,36.650000,3.420000,829.0,60.0,NaN,...,1976.0,1979.0,4.0,0.0,1976,1979,4,0.354,0.011164,13.466519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6987100,6,6870,SHAKHE,SOLOKHAUL,RU,43.799444,39.678056,423.0,133.0,1978.0,...,1978.0,1987.0,10.0,0.0,1978,1987,10,29.333,0.925045,2186.868766
6987150,6,6870,SOCHI,PLASTUNKA,RU,43.680556,39.822222,238.0,-999.0,1978.0,...,1978.0,1987.0,10.0,0.0,1978,1987,10,15.585,0.491489,2065.077983
6998200,6,6984,KANGERLUARSUNNGUUP KUUA,KOBBEFJORD HYDROMETRIC STATION,DK,64.133110,-51.380780,31.0,22.0,2006.0,...,NaN,NaN,NaN,NaN,2006,2020,15,1.219,0.038442,1240.076903


In [5]:
# read the data
data_base_path = '/data/compoundx/causal_flood/stability_testing/data/'
all_peaks = {}
all_data = {}
for dataset_context in tqdm(['observed','simulated','resampled']):
    data_path = os.path.join(data_base_path,dataset_context)
    all_peaks[dataset_context] = {}
    all_data[dataset_context] = {}
    for id in tqdm(ids):
        peak_indices = []
        with open(os.path.join(data_path, f'{id}_peak_indices_USWRC.txt'), 'r') as filehandle:
            peak_indices = [current_place.rstrip() for current_place in filehandle.readlines()]
        all_peaks[dataset_context][id] = [int(p)for p in peak_indices]
        data = pd.read_csv(os.path.join(data_path, f'{id}.csv'))
        data['time'] = data['time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
        all_data[dataset_context][id] = data



100%|██████████| 3/3 [01:56<00:00, 38.98s/it]


# Analysis

In [34]:
all_data['simulated'][6337610]

,time,tavg,pre,snow,SM,Q,year
0,1950-01-01,-3.805588,0.000000,15.000000,0.362190,16.199885,1950
1,1950-01-02,-1.557454,8.462839,19.058369,0.362170,22.295450,1950
2,1950-01-03,1.283638,4.136272,23.273736,0.365218,21.474517,1950
3,1950-01-04,-0.731928,2.827923,24.611291,0.368517,20.487493,1950
4,1950-01-05,0.462693,4.840284,27.492663,0.369857,19.729577,1950
...,...,...,...,...,...,...,...
26293,2021-12-27,0.470614,1.009686,9.176982,0.976315,9.919044,2021
26294,2021-12-28,3.244654,9.231579,5.913654,0.979230,13.259252,2021
26295,2021-12-29,5.516405,5.927967,2.829954,0.982039,20.973894,2021
26296,2021-12-30,9.398890,3.573850,0.474050,0.983295,28.235926,2021


## Metrics

In [30]:
def number_of_peaks(peaks:list):
    """
    Returns the number peaks.
    """
    return len(peaks)

def count_non_nan_q(df):
    """
    Returns the number of data points where Q is not NaN.
    """
    return df['Q'].notna().sum()

def first_date_with_data(df):
    """
    Returns the first date where Q is not NaN.
    """
    return df.loc[df['Q'].notna(), 'time'].min()

def last_date_with_data(df):
    """
    Returns the last date where Q is not NaN.
    """
    return df.loc[df['Q'].notna(), 'time'].max()

def mean_non_nan_q(df):
    """
    Returns the mean of non-NaN values in the Q column.
    """
    return df['Q'].mean()

def std_dev_non_nan_q(df):
    """
    Returns the standard deviation of non-NaN values in the Q column.
    """
    return df['Q'].std()

def calculate_nse(obs_df, sim_df):
    """
    Calculate the Nash-Sutcliffe Efficiency (NSE) between observed and simulated data.
    """
    # Merge the two DataFrames on the 'time' column
    merged_df = pd.merge(obs_df, sim_df, on='time', suffixes=('_obs', '_sim'))

    # Drop rows where either Q_obs or Q_sim is NaN
    merged_df = merged_df.dropna(subset=['Q_obs', 'Q_sim'])

    # Extract observed and simulated values
    Q_obs = merged_df['Q_obs']
    Q_sim = merged_df['Q_sim']

    # Calculate NSE
    numerator = np.sum((Q_obs - Q_sim) ** 2)
    denominator = np.sum((Q_obs - np.mean(Q_obs)) ** 2)
    nse = 1 - (numerator / denominator)

    return nse

def calculate_kge(obs_df, sim_df):
    """
    Calculate the Kling-Gupta Efficiency (KGE) between observed and simulated data.
    """
    # Merge the two DataFrames on the 'time' column
    merged_df = pd.merge(obs_df, sim_df, on='time', suffixes=('_obs', '_sim'))

    # Drop rows where either Q_obs or Q_sim is NaN
    merged_df = merged_df.dropna(subset=['Q_obs', 'Q_sim'])

    # Extract observed and simulated values
    Q_obs = merged_df['Q_obs']
    Q_sim = merged_df['Q_sim']

    # Calculate components of KGE
    r = np.corrcoef(Q_obs, Q_sim)[0, 1]  # Pearson correlation coefficient
    alpha = np.std(Q_sim) / np.std(Q_obs)  # Ratio of standard deviations
    beta = np.mean(Q_sim) / np.mean(Q_obs)  # Ratio of means

    # Calculate KGE
    kge = 1 - np.sqrt((r - 1) ** 2 + (alpha - 1) ** 2 + (beta - 1) ** 2)

    return kge

def mean_q_at_peaks(df, peak_indices):
    """
    Calculate the mean value of Q at the given peak indices.
    """
    # Extract Q values at the peak indices
    q_values_at_peaks = df.loc[peak_indices, 'Q']

    # Calculate the mean of the Q values at the peaks
    mean_q = q_values_at_peaks.mean()

    return mean_q

def mean_annual_sum_pre(df):
    """
    Calculate the annual sum of the 'pre' column.
    """
    # Extract the year directly from the datetime objects in the 'time' column
    df['year'] = df['time'].apply(lambda x: x.year)

    # Group by year and calculate the sum of 'pre'
    annual_sum = df.groupby('year')['pre'].sum()

    return annual_sum.mean()

def avg_pre_values_above_1(df):
    """
    Calculate the average number of values below 1 in the 'pre' column per year.
    """
    # Extract the year directly from the datetime objects in the 'time' column
    df['year'] = df['time'].apply(lambda x: x.year)

    # Count the number of values below 1 for each year
    values_below_1 = df[df['pre'] > 1].groupby('year').size()

    # Calculate the average number of values below 1 across all years
    avg_below_1 = values_below_1.mean()

    return avg_below_1

def mean_tavg(df):
    """
    Calculate the mean of the 'tavg' column.
    """
    return df['tavg'].mean()

def std_dev_tavg(df):
    """
    Calculate the standard deviation of the 'tavg' column.
    """
    return df['tavg'].std()


def mean_days_snow_not_zero(df):
    """
    Calculate the mean number of days per year where 'snow' is above 1.
    """
    # Extract the year directly from the datetime objects in the 'time' column
    df['year'] = df['time'].apply(lambda x: x.year)

    # Count the number of days per year where 'snow' is above 1
    days_snow_above_1 = df[df['snow'] > 0].groupby('year').size()

    # Calculate the mean number of days across all years
    mean_days = days_snow_above_1.mean()

    return mean_days

def mean_snow_not_zero(df):
    """
    Calculate the mean of the 'snow' column, excluding values that are 0.

    Parameters:
    - df: pandas DataFrame with a column 'snow'.

    Returns:
    - The mean of 'snow' when it is not 0.
    """
    # Filter out rows where 'snow' is 0 and calculate the mean
    mean_snow = df[df['snow'] > 0]['snow'].mean()

    return mean_snow

def mean_sm(df):
    """
    Calculate the mean of the 'SM' column.
    """
    return df['SM'].mean()

def std_dev_sm(df):
    """
    Calculate the standard deviation of the 'SM' column.
    """
    return df['SM'].std()

def cut_off_string(s):
    # Find the index of the first comma or parenthesis
    comma_index = s.find(',')
    paren_index = s.find('(')

    # Determine the earliest index to cut off
    if comma_index == -1 and paren_index == -1:
        return s  # No comma or parenthesis found, return the whole string
    elif comma_index == -1:
        return s[:paren_index]  # Only parenthesis found
    elif paren_index == -1:
        return s[:comma_index]  # Only comma found
    else:
        return s[:min(comma_index, paren_index)]  # Both found, take the earliest one

def all_basin_statistics(id, data_observed, data_simulated, data_resampled, peaks_observed, peaks_simulated, peaks_resampled, country = None, river = None, station = None, area = None):
    """
    Calculate all basin statistics for a basin
    """
    results = {}
    results[('Basin','id','$ $')] = id
    results[('Basin', 'county','$ $')] = country
    results[('Basin', 'river','$ $')] = cut_off_string(river)
    results[('Basin', 'station','$ $')] = cut_off_string(station)
    results[('Basin', 'area [$km^2$]','$ $')] = area


    results[('Observations', 'start', 'date')] = first_date_with_data(data_observed)
    results[('Observations', 'end','date')] = last_date_with_data(data_observed)
    results[('Observations', 'days','count')] = count_non_nan_q(data_observed)
    results[('Observations', 'peaks','count')] = number_of_peaks(peaks_observed)
    results[('Observations', 'discharge [$m^3$/s]', 'mean')] = mean_non_nan_q(data_observed)
    results[('Observations','discharge [$m^3$/s]', 'SD')] = std_dev_non_nan_q(data_observed)
    results[('Observations','discharge [$m^3$/s]', 'mean peak')] = mean_q_at_peaks(data_observed, peaks_observed)
    results[('Observations', 'precipitation [mm]', 'mas')] = mean_annual_sum_pre(data_observed) # mean annual sum
    results[('Observations', 'precipitation [mm]', 'mada1')] =  avg_pre_values_above_1(data_observed) # mean annual days above 1mm
    results[('Observations', 'temperature [°C]', 'mean')] = mean_tavg(data_observed)
    results[('Observations', 'temperature [°C]', 'SD')] = std_dev_tavg(data_observed)
    results[('Observations', 'snow [mm]', 'mada0')] = mean_days_snow_not_zero(data_observed) # mean annual days above 0
    # results[('Observations', 'snow [mm]', 'mean snow on days with snow')] = mean_snow_not_zero(data_observed)
    results[('Observations', 'soil moisture [\%]', 'mean')] = mean_sm(data_observed)
    results[('Observations', 'soil moisture [\%]', 'SD')] = std_dev_sm(data_observed)
    
    results[('Simulations', 'model performance', 'NSE')] = calculate_nse(data_observed, data_simulated)
    results[('Simulations', 'model performance', 'KGE')] = calculate_kge(data_observed, data_simulated)
    results[('Simulations', 'peaks','count')] = number_of_peaks(peaks_simulated)
    results[('Simulations', 'discharge [$m^3$/s]', 'mean')] = mean_non_nan_q(data_simulated)
    results[('Simulations','discharge [$m^3$/s]', 'SD')] = std_dev_non_nan_q(data_simulated)
    results[('Simulations','discharge [$m^3$/s]', 'mean peak')] = mean_q_at_peaks(data_simulated, peaks_simulated)
    results[('Simulations', 'precipitation [mm]', 'mas')] = mean_annual_sum_pre(data_simulated) # mean annual sum
    results[('Simulations', 'precipitation [mm]', 'mada1')] =  avg_pre_values_above_1(data_simulated) # mean annual days above 1mm
    results[('Simulations', 'temperature [°C]', 'mean')] = mean_tavg(data_simulated)
    results[('Simulations', 'temperature [°C]', 'SD')] = std_dev_tavg(data_simulated)
    results[('Simulations', 'snow [mm]', 'mada0')] = mean_days_snow_not_zero(data_simulated) # mean annual days above 0
    # results[('Simulations', 'snow [mm]', 'mean snow on days with snow')] = mean_snow_not_zero(data_simulated)
    results[('Simulations', 'soil moisture [\%]', 'mean')] = mean_sm(data_simulated)
    results[('Simulations', 'soil moisture [\%]', 'SD')] = std_dev_sm(data_simulated)

    results[('Resampled', 'peaks','count')] = number_of_peaks(peaks_resampled)
    results[('Resampled', 'discharge [$m^3$/s]', 'mean')] = mean_non_nan_q(data_resampled)
    results[('Resampled','discharge [$m^3$/s]', 'SD')] = std_dev_non_nan_q(data_resampled)
    results[('Resampled','discharge [$m^3$/s]', 'mean peak')] = mean_q_at_peaks(data_resampled, peaks_resampled)
    results[('Resampled', 'precipitation [mm]', 'mas')] = mean_annual_sum_pre(data_resampled) # mean annual sum
    results[('Resampled', 'precipitation [mm]', 'mada1')] =  avg_pre_values_above_1(data_resampled) # mean annual days above 1mm
    results[('Resampled', 'temperature [°C]', 'mean')] = mean_tavg(data_resampled)
    results[('Resampled', 'temperature [°C]', 'SD')] = std_dev_tavg(data_resampled)
    results[('Resampled', 'snow [mm]', 'mada0')] = mean_days_snow_not_zero(data_resampled) # mean annual days above 0
    # results[('Resampled', 'snow [mm]', 'mean snow on days with snow')] = mean_snow_not_zero(data_resampled)
    results[('Resampled', 'soil moisture [\%]', 'mean')] = mean_sm(data_resampled)
    results[('Resampled', 'soil moisture [\%]', 'SD')] = std_dev_sm(data_resampled)


    
    # results['Simulations peaks'] = number_of_peaks(peaks_simulated)
    # results['Simulations mean Q'] = mean_non_nan_q(data_simulated)
    # results['Simulations std. dev. Q'] = std_dev_non_nan_q(data_simulated)
    # results['Simulations mean peak Q'] = mean_q_at_peaks(data_simulated, peaks_simulated)
    

    # results['Resampled Peaks'] = number_of_peaks(peaks_resampled)


    # Convert the dictionary to a DataFrame with multi-level columns
    df = pd.DataFrame.from_dict({k: [v] for k, v in results.items()}, orient='columns')

    # Set the multi-level column headers
    df.columns = pd.MultiIndex.from_tuples(df.columns)

    return df


In [31]:
combined_results = pd.DataFrame()
statistics_results = []
for id in ids:
    statistics_results.append(all_basin_statistics(
        id = id,
        data_observed=all_data['observed'][id],
        data_simulated=all_data['simulated'][id],
        data_resampled=all_data['resampled'][id],
        peaks_observed=all_peaks['observed'][id],
        peaks_simulated=all_peaks['simulated'][id],
        peaks_resampled=all_peaks['resampled'][id],
        country = grdc_info['country'].loc[id],
        river = grdc_info['river'].loc[id],
        station = grdc_info['station'].loc[id],
        area = grdc_info['area'].loc[id],
        ))
    combined_results = pd.concat(statistics_results, axis=0).reset_index(drop=True)

    

In [32]:
combined_results

Basin                                                              \
         id county           river                station area [$km^2$]   
        $ $    $ $             $ $                    $ $           $ $   
0   6337610     DE           WERRA              MEININGEN        1170.0   
1   6502171     IE     BOYLE RIVER              TINACARRA         520.0   
2   6335046     DE            SIEG                 EITORF        1472.0   
3   6338200     DE          VECHTE             EMLICHHEIM        1687.0   
4   6338160     DE            HASE                BOKELOH        2968.0   
5   6123710     FR          ARROUX       RIGNY-SUR-ARROUX        2277.0   
6   6503851     IE           BOYNE           SLANE CASTLE        2408.0   
7   6229100     SE  ENNINGDALSELVA             VASSBOTTEN         624.1   
8   6335351     DE            LAHN                MARBURG        1667.0   
9   6123350     FR         SAULDRE        SELLES-SUR-CHER        2254.0   
10  6139682     FR           OGNON                 PESMES        2038.0   
11  6503280     IE            SUIR                CLONMEL        2143.9   
12  6503500     IE      BLACKWATER              BALLYDUFF        2338.0   
13  6731600     NO          VEFSNA              UNKERVATN         756.0   
14  6338120     DE       EMS RIVER                 GREVEN        2842.0   
15  6934571     DK            SUSA        HOLLOESE MOELLE         756.0   
16  6338161     DE            HASE               HERZLAKE        2218.0   
17  6503855     IE      BLACKWATER              LISCARTON         717.0   
18  6335160     DE           NIDDA             ILBENSTADT        1072.0   
19  6139790     FR            LOUE     CHAMPAGNE-SUR-LOUE        1380.0   
20  6502151     IE            SUCK            DERRYCAHILL        1050.0   
21  6233350     SE        ALSTERAN                GETEBRO        1332.7   
22  6233100     SE          VISKAN                ASBRO 3        2160.2   
23  6321100     DE           NIERS                   GOCH        1203.0   
24  6342521     DE         ALTMUHL  BEILNGRIES ABOVE SULZ        2475.0   
25  6604220     GB           LEVEN              LINNBRANE         784.3   
26  6243400     AT   OTZTALER ACHE                 TUMPEN         785.5   
27  6338150     DE       EMS RIVER                  EINEN        1499.0   
28  6503281     IE            SUIR             CAHIR PARK        1602.0   
29  6337504     DE            EDER         SCHMITTLOTHEIM        1202.0   
30  6335045     DE            SIEG               MENDEN 1        2832.0   
31  6503201     IE            ERNE              BELTURBET        1501.0   
32  6335081     DE           LIPPE             KESSELER 3        2005.0   
33  6119200     FR           LEYRE                 SALLES        1650.0   
34  6854590     FI        OULUJOKI     LAKE LENTUA OUTLET        2045.0   
35  6337050     DE           HUNTE            COLNRADE OP        1313.0   
36  6503180     IE          FERGUS             BALLYCOREY         562.0   
37  6123160     FR  SEVRE NANTAISE             TIFFAUGES          814.0   
38  6503351     IE          BARROW              ROYAL OAK        2415.0   
39  6335360     DE            DILL                 ASSLAR         692.2   
40  6503301     IE            NORE           JOHNS BRIDGE        1605.0   
41  6855409     FI          VUOKSI              HAAPAJOKI         596.0   
42  6136200     FR   MOSELLE RIVER                 EPINAL        1220.0   
43  6503300     IE            NORE             BROWNSBARN        2388.0   
44  6233520     SE         TARNAAN                SOLBERG        1083.5   

   Observations                                              ...  \
          start        end   days peaks discharge [$m^3$/s]  ...   
           date       date  count count                mean  ...   
0    1950-01-01 2019-08-31  25445   863           14.307174  ...   
1    1957-10-24 2002-01-21  16000   281           11.637879  ...   
2    1967-11-01 2018-10-31  18625   838           27.569198  ... 

In [33]:

def export_dataframe(df, filename_prefix):
    """
    Exports a multi-index DataFrame to CSV, HTML, and LaTeX formats.
    """
    # round to 1 digit to improve readibility
    df = df.round(2)
    # Export to CSV
    csv_filename = f"{filename_prefix}.csv"
    df.to_csv(csv_filename, index=False)
    print(f"Exported to CSV: {csv_filename}")

    # Export to HTML
    html_filename = f"{filename_prefix}.html"
    df.to_html(html_filename, index=False)
    print(f"Exported to HTML: {html_filename}")

    # Export to LaTeX
    latex_filename = f"{filename_prefix}.tex"
    df.to_latex(latex_filename, index=False, escape=False)
    print(f"Exported to LaTeX: {latex_filename}")


export_dataframe(combined_results, os.path.join(data_base_path,'summary_statistics'))

Exported to CSV: /data/compoundx/causal_flood/stability_testing/data/summary_statistics.csv
Exported to HTML: /data/compoundx/causal_flood/stability_testing/data/summary_statistics.html
Exported to LaTeX: /data/compoundx/causal_flood/stability_testing/data/summary_statistics.tex


/tmp/ipykernel_433765/3270407684.py:19: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.to_latex(latex_filename, index=False, escape=False)
